In [ ]:
#!pip uninstall 'qiskit-aqua' -y
!pip install --upgrade 'qiskit[visualization,optimization]'

In [10]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import Aer, IBMQ

from qiskit.opflow import PauliExpectation, CVaRExpectation


from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import GroverOptimizer 

from qiskit.tools.jupyter import *

from qiskit.utils import algorithm_globals
algorithm_globals.random_seed = 123456
algorithm_globals.massive=True


import numpy as np
import math
from docplex.mp.model import Model
from math import log

# Loading your IBM Q account(s)
provider = IBMQ.load_account()

ibmqfactory.load_account:WARNING:2021-04-15 11:25:02,910: Credentials are already in use. The existing account in the session will be replaced.


# ETF Optimization (Use_case_v3) 


## Binary representation of integers limited to budget amount

(Based on H2 Hypothesis in paper v1 and stock grouping in paper v2)

$\begin{aligned}
\min_{x \in \mathbb{N}^n}  q x^T \Sigma x - \mu^T x\\
\end{aligned}$

subject to:

$\begin{aligned}
\ \sum_{i=0}^{n} S^T x \le B,\quad x \in \mathbb{N}^n,\quad S \in \mathbb{R}^n,\quad B \in \mathbb{R} \qquad \approx_{k\to\infty}  \qquad \sum_{i=0}^{n}  ceil( \frac{k}{\overline{S}}s_i) x_i \le floor(\frac{k}{\overline{S}}B) ,\quad k \in \mathbb{R},\quad ceil(.), floor(.) \in \mathbb{N}\\
\end{aligned}$

integer variables $x_i$ bounded to:

$\begin{aligned}
0 \le x_i \le 2^{floor(log_{2}(\frac{B}{\min_{j}{s_j}}))}-1
\end{aligned}$

where we use the following notation:

- $x \in \mathbb{N}^n$ denotes the vector of integer decision variables, which indicate which assets to pick and how much ($x[i] > 0$) and which not to pick ($x[i] = 0$),
- $y$ represents the binary decomposition of $x$
- $\mu \in \mathbb{R}^n$ defines the expected returns for the assets,
- $\Sigma \in \mathbb{R}^{n \times n}$ specifies the covariances between the assets,
- $q > 0$ controls the risk appetite of the decision maker,
- $S$ defined as the prices related to the (grouped) stocks ( $\overline{S}$ is the mean of the prices),
- $B$ denotes the budget, i.e. the maximum amount of money to be spent purchasing assets.
- $ceil$ is the operator that rounds up to the nearest integer
- $floor$ is the operator that rounds down to the nearest integer
- $k$ is a scaling factor used to tune the precision of the conversions to integer


In [2]:
# prepare problem instance
n = 5            # number of assets
q = 0.5          # risk factor
budget = 2250  # budget

In [3]:

# ETF actual prices
s_real = np.array([60.04, 55.87, 139.72, 32.46, 111.07])
grouping = np.array([2, 3, 1, 4, 1])
s=grouping*s_real

print(' ̃s:', s)

 ̃s: [120.08 167.61 139.72 129.84 111.07]


In [4]:
print(s.mean())

133.664


In [5]:
budget_bits = int(np.log2(budget/np.min(s))) #TODO: int rounds down. I guess this is just excluding high volatility solutions.
print("Budget variables per asset:", budget_bits)

Budget variables per asset: 4


In [6]:
mu = np.array([0.00290, 0.00206, 0.00033, 0.00263, -0.00012])
sigma = np.array([
    [ 0.00016, 0.00014, 0.00013, 0.00016, 0.00015],
    [ 0.00014, 0.00016, 0.00013, 0.00016, 0.00014],
    [ 0.00013, 0.00013, 0.00028, 0.00013, 0.00028],
    [ 0.00016, 0.00016, 0.00013, 0.00021, 0.00015],
    [ 0.00015, 0.00014, 0.00028, 0.00015, 0.00030]
])

In [7]:
# create docplex model
mdl = Model('portfolio_optimization')

x = mdl.integer_var_list((f'x{i}' for i in range(n)), lb=0, ub=2**budget_bits-1)

objective = mdl.sum([mu[i]*x[i] for i in range(n)])
objective -= q * mdl.sum([sigma[i,j]*x[i]*x[j] for i in range(n) for j in range(n)])
mdl.maximize(objective)

k = 7
norm = s.mean()/k
mdl.add_constraint(mdl.sum(x[i] * math.ceil(s[i]/norm) for i in range(n)) <= math.floor(budget/norm))


docplex.mp.LinearConstraint[](7x0+9x1+8x2+7x3+6x4,LE,117)

In [8]:
qp = QuadraticProgram()
qp.from_docplex(mdl)

print('### Original problem:')
print(qp.export_as_lp_string())

### Original problem:
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: portfolio_optimization

Maximize
 obj: 0.002900000000 x0 + 0.002060000000 x1 + 0.000330000000 x2
      + 0.002630000000 x3 - 0.000120000000 x4 + [ - 0.000160000000 x0^2
      - 0.000280000000 x0*x1 - 0.000260000000 x0*x2 - 0.000320000000 x0*x3
      - 0.000300000000 x0*x4 - 0.000160000000 x1^2 - 0.000260000000 x1*x2
      - 0.000320000000 x1*x3 - 0.000280000000 x1*x4 - 0.000280000000 x2^2
      - 0.000260000000 x2*x3 - 0.000560000000 x2*x4 - 0.000210000000 x3^2
      - 0.000300000000 x3*x4 - 0.000300000000 x4^2 ]/2
Subject To
 c0: 7 x0 + 9 x1 + 8 x2 + 7 x3 + 6 x4 <= 117

Bounds
       x0 <= 15
       x1 <= 15
       x2 <= 15
       x3 <= 15
       x4 <= 15

Generals
 x0 x1 x2 x3 x4
End



In [14]:
backend = Aer.get_backend('qasm_simulator')

cop = GroverOptimizer(num_value_qubits= 3, num_iterations = 3, quantum_instance=backend)
classical_result = cop.solve(qp)

In [ ]:
conv = QuadraticProgramToQubo()
qp1 = conv.convert(qp)

print('### quadratic_program_to_qubo:')
print(qp1.export_as_lp_string())
print("Penalty:", conv.penalty)

In [ ]:
# Call the aggregator with 'optimizer' as the algorithm of choice
results = aggregator('optimizer', optim_dict)

In [ ]:
results['result'].variables_dict

In [ ]:
print()
print('Integer results (amount of groups of stocks): x')
x_val = [results['result'].variables_dict[f'x{i}'] for i in range(5)]
print(x_val)

print()
print('Amount of individual stock:')
x_ind = grouping*x_val
print(x_ind)

print()
print('Amount invest in each stock:')
invest = s_real*x_ind
print(invest)

print()
print('Total invest:')
print(invest.sum())

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table

In [ ]:
qiskit.__qiskit_version__

In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table

In [ ]:
qiskit.__qiskit_version__